<a href="https://colab.research.google.com/github/andryll/Cortisol-Estimator-with-DL/blob/main/notebooks/CortIncepTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
pip install aeon

## Carregando o repositório do GitHub com os dados

In [2]:
!git clone https://github.com/andryll/Cortisol-Estimator-with-DL

Cloning into 'Cortisol-Estimator-with-DL'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 2.14 MiB | 4.15 MiB/s, done.
Resolving deltas: 100% (9/9), done.


## Carregando e Preparando os Dados


1.   Os dados de treino, teste e validação são carregados e concatenados, de modo a desfazer a divisão feita anteriormente.

2.   Em seguida, os dados de teste são normalizados com o MinMaxScaler



In [3]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

data_X = np.load("/content/Cortisol-Estimator-with-DL/data/X_train.npy")
data_X = np.concatenate((data_X, np.load("/content/Cortisol-Estimator-with-DL/data/X_val.npy")), axis=0)
data_X = np.concatenate((data_X, np.load("/content/Cortisol-Estimator-with-DL/data/X_test.npy")), axis=0)
print("data_X shape:", data_X.shape)

data_y = np.load("/content/Cortisol-Estimator-with-DL/data/y_train.npy")
data_y = np.concatenate((data_y, np.load("/content/Cortisol-Estimator-with-DL/data/y_val.npy")), axis=0)
data_y = np.concatenate((data_y, np.load("/content/Cortisol-Estimator-with-DL/data/y_test.npy")), axis=0)

scaler = MinMaxScaler()
data_y = scaler.fit_transform(data_y.reshape(-1, 1)).flatten()

print("data_y shape:", data_y.shape)

data_X shape: (148, 587, 4)
data_y shape: (148,)


## Realizando a Regressão com o InceptionTime

In [7]:
from aeon.regression.deep_learning import InceptionTimeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Realizando o KFold split com 10 folds
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

mses = [] # Lista para armazenar o MSE de cada Fold

print(f"Realizando o {n_splits}-fold cross-validation...")

# Loop iterando cada uma das pastas
for fold, (train_index, test_index) in enumerate(kf.split(data_X)): # Enumerando as pastas
    print(f"\nFold {fold+1}/{n_splits}")

    # Dividindo os dados de acordo com a pasta atual
    X_train_fold, X_test_fold = data_X[train_index], data_X[test_index]
    y_train_fold, y_test_fold = data_y[train_index], data_y[test_index]

    # Inicializando o InceptionTimeRegressor com 30 épocas e treino ao vivo
    reg = InceptionTimeRegressor(n_epochs = 30, batch_size = 1)
    # Treinando o Regressor
    reg.fit(X_train_fold, y_train_fold)

    # Realizando as predições
    y_pred_fold = reg.predict(X_test_fold)

    # Calculando o MSE para a pasta atual
    mse_fold = mean_squared_error(y_test_fold, y_pred_fold)
    mses.append(mse_fold)
    print(f"MSE para a pasta {fold+1}: {mse_fold}")

# Calculando a média dos MSE
average_mse = np.mean(mses)
print(f"\nAverage MSE across {n_splits} folds: {average_mse}")

Realizando o 10-fold cross-validation...

Fold 1/10


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 479ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step
MSE para a pasta 1: 0.03649491066368915

Fold 2/10


KeyboardInterrupt: 